In [ ]:
import pymongo

In [ ]:
MONGO_DB = "bsocoverage"
MONGO_COLLECTION = "publications"

In [ ]:
mongo_database = pymongo.MongoClient()[MONGO_DB]
mongo_collection = mongo_database[MONGO_COLLECTION]

In [ ]:
# Check publication year consistency accross FOSM and OpenAlex
print('id,year_fosm,year_openalex')
for publication in mongo_collection.find():
    if publication.get("year_fosm") and publication.get("year_openalex") and publication.get("year_fosm") != publication.get("year_openalex"):
        print(f"{publication.get('id')},{publication.get('year_fosm')},{publication.get('year_openalex')}")
# See errors in errors_publication_year.csv

In [ ]:
actions = []
for publication in mongo_collection.find():
  if publication.get("year_fosm"):
    actions.append(pymongo.UpdateOne({ "id": publication.get("id") }, { "$set": { "year": publication.get("year_fosm") } }, upsert=True))
  elif publication.get("year_openalex"):
    actions.append(pymongo.UpdateOne({ "id": publication.get("id") }, { "$set": { "year": publication.get("year_openalex") } }, upsert=True))
if len(actions) > 0:
  mongo_collection.bulk_write(actions, ordered=False)